##getting the data

In [ ]:
from google.colab import drive
drive.mount("/content/drive")

Mounted at /content/drive


In [ ]:
import numpy as np
import pandas as pd

In [ ]:
data=pd.read_csv("/content/drive/MyDrive/GenDis/train_genetic_disorders.csv")

##dropping least relevant columns

In [ ]:
data.drop(["Test 1", "Test 2", "Test 3", "Test 4", "Test 5", "Family Name", "Patient First Name", "Father's name", "Institute Name", "Location of Institute", "Autopsy shows birth defect (if applicable)", "Parental consent"],inplace=True,axis=1)

In [ ]:
data = data.rename(columns={'Inherited from father': 'Genes in father\'s side'})
# data.head()

In [ ]:
data.shape

(22083, 33)

##performing encoding for categorical data

In [ ]:
data["Birth asphyxia"].replace({"Not available": "No record"}, inplace=True)
data["H/O radiation exposure (x-ray)"].replace({"-": np.nan}, inplace=True)
data["H/O substance abuse"].replace({"-": np.nan}, inplace=True)
data["Blood test result"].replace({"inconclusive": 0, "normal": 1, "slightly abnormal": 2, "abnormal": 3}, inplace=True)

In [ ]:
data['Genes in mother\'s side'] = data['Genes in mother\'s side'].astype('category')
data['Genes in mother\'s side'] = data['Genes in mother\'s side'].cat.codes
data['Genes in father\'s side'] = data['Genes in father\'s side'].astype('category')
data['Genes in father\'s side'] = data['Genes in father\'s side'].cat.codes
data['Maternal gene'] = data['Maternal gene'].astype('category')
data['Maternal gene'] = data['Maternal gene'].cat.codes
data['Paternal gene'] = data['Paternal gene'].astype('category')
data['Paternal gene'] = data['Paternal gene'].cat.codes
data['Status'] = data['Status'].astype('category')
data['Status'] = data['Status'].cat.codes
data['Respiratory Rate (breaths/min)'] = data['Respiratory Rate (breaths/min)'].astype('category')
data['Respiratory Rate (breaths/min)'] = data['Respiratory Rate (breaths/min)'].cat.codes
data['Heart Rate (rates/min'] = data['Heart Rate (rates/min'].astype('category')
data['Heart Rate (rates/min'] = data['Heart Rate (rates/min'].cat.codes
data['Follow-up'] = data['Follow-up'].astype('category')
data['Follow-up'] = data['Follow-up'].cat.codes
data['Gender'] = data['Gender'].astype('category')
data['Gender'] = data['Gender'].cat.codes
data['Birth asphyxia'] = data['Birth asphyxia'].astype('category')
data['Birth asphyxia'] = data['Birth asphyxia'].cat.codes
data['Place of birth'] = data['Place of birth'].astype('category')
data['Place of birth'] = data['Place of birth'].cat.codes
data['Folic acid details (peri-conceptional)'] = data['Folic acid details (peri-conceptional)'].astype('category')
data['Folic acid details (peri-conceptional)'] = data['Folic acid details (peri-conceptional)'].cat.codes
data['H/O serious maternal illness'] = data['H/O serious maternal illness'].astype('category')
data['H/O serious maternal illness'] = data['H/O serious maternal illness'].cat.codes
data['H/O radiation exposure (x-ray)'] = data['H/O radiation exposure (x-ray)'].astype('category')
data['H/O radiation exposure (x-ray)'] = data['H/O radiation exposure (x-ray)'].cat.codes
data['H/O substance abuse'] = data['H/O substance abuse'].astype('category')
data['H/O substance abuse'] = data['H/O substance abuse'].cat.codes
data['Assisted conception IVF/ART'] = data['Assisted conception IVF/ART'].astype('category')
data['Assisted conception IVF/ART'] = data['Assisted conception IVF/ART'].cat.codes
data['History of anomalies in previous pregnancies'] = data['History of anomalies in previous pregnancies'].astype('category')
data['History of anomalies in previous pregnancies'] = data['History of anomalies in previous pregnancies'].cat.codes
data['Birth defects'] = data['Birth defects'].astype('category')
data['Birth defects'] = data['Birth defects'].cat.codes
data['Genetic Disorder'] = data['Genetic Disorder'].astype('category')
data['Genetic Disorder'] = data['Genetic Disorder'].cat.codes
data['Disorder Subclass'] = data['Disorder Subclass'].astype('category')
data['Disorder Subclass'] = data['Disorder Subclass'].cat.codes
data.head()

,Patient Id,Patient Age,Genes in mother's side,Genes in father's side,Maternal gene,Paternal gene,Blood cell count (mcL),Mother's age,Father's age,Status,Respiratory Rate (breaths/min),Heart Rate (rates/min,Follow-up,Gender,Birth asphyxia,Place of birth,Folic acid details (peri-conceptional),H/O serious maternal illness,H/O radiation exposure (x-ray),H/O substance abuse,Assisted conception IVF/ART,History of anomalies in previous pregnancies,No. of previous abortion,Birth defects,White Blood cell count (thousand per microliter),Blood test result,Symptom 1,Symptom 2,Symptom 3,Symptom 4,Symptom 5,Genetic Disorder,Disorder Subclass
0,PID0x6418,2.0,1,0,1,0,4.760603,NaN,NaN,0,0,0,0,-1,-1,1,0,-1,0,0,0,1,NaN,-1,9.857562,NaN,1.0,1.0,1.0,1.0,1.0,0,5
1,PID0x25d5,4.0,1,1,0,0,4.910669,NaN,23.0,1,1,0,0,-1,0,-1,1,1,1,1,0,1,NaN,0,5.522560,1.0,1.0,NaN,1.0,1.0,0.0,-1,2
2,PID0x4a82,6.0,1,0,0,0,4.893297,41.0,22.0,0,0,1,1,-1,1,-1,1,0,2,-1,1,1,4.0,1,NaN,1.0,0.0,1.0,1.0,1.0,1.0,1,3
3,PID0x4ac8,12.0,1,0,1,0,4.705280,21.0,NaN,1,1,0,0,2,1,1,0,1,-1,1,-1,1,1.0,1,7.919321,0.0,0.0,0.0,1.0,0.0,0.0,0,6
4,PID0x1bf7,11.0,1,0,-1,1,4.720703,32.0,NaN,0,1,1,1,2,1,1,0,1,-1,1,1,0,4.0,0,4.098210,NaN,0.0,0.0,0.0,0.0,NaN,1,1


In [ ]:
for i in data.columns:
  data[i].replace({-1: np.nan}, inplace=True)
data.head()

,Patient Id,Patient Age,Genes in mother's side,Genes in father's side,Maternal gene,Paternal gene,Blood cell count (mcL),Mother's age,Father's age,Status,Respiratory Rate (breaths/min),Heart Rate (rates/min,Follow-up,Gender,Birth asphyxia,Place of birth,Folic acid details (peri-conceptional),H/O serious maternal illness,H/O radiation exposure (x-ray),H/O substance abuse,Assisted conception IVF/ART,History of anomalies in previous pregnancies,No. of previous abortion,Birth defects,White Blood cell count (thousand per microliter),Blood test result,Symptom 1,Symptom 2,Symptom 3,Symptom 4,Symptom 5,Genetic Disorder,Disorder Subclass
0,PID0x6418,2.0,1.0,0.0,1.0,0.0,4.760603,NaN,NaN,0.0,0.0,0.0,0.0,NaN,NaN,1.0,0.0,NaN,0.0,0.0,0.0,1.0,NaN,NaN,9.857562,NaN,1.0,1.0,1.0,1.0,1.0,0.0,5.0
1,PID0x25d5,4.0,1.0,1.0,0.0,0.0,4.910669,NaN,23.0,1.0,1.0,0.0,0.0,NaN,0.0,NaN,1.0,1.0,1.0,1.0,0.0,1.0,NaN,0.0,5.522560,1.0,1.0,NaN,1.0,1.0,0.0,NaN,2.0
2,PID0x4a82,6.0,1.0,0.0,0.0,0.0,4.893297,41.0,22.0,0.0,0.0,1.0,1.0,NaN,1.0,NaN,1.0,0.0,2.0,NaN,1.0,1.0,4.0,1.0,NaN,1.0,0.0,1.0,1.0,1.0,1.0,1.0,3.0
3,PID0x4ac8,12.0,1.0,0.0,1.0,0.0,4.705280,21.0,NaN,1.0,1.0,0.0,0.0,2.0,1.0,1.0,0.0,1.0,NaN,1.0,NaN,1.0,1.0,1.0,7.919321,0.0,0.0,0.0,1.0,0.0,0.0,0.0,6.0
4,PID0x1bf7,11.0,1.0,0.0,NaN,1.0,4.720703,32.0,NaN,0.0,1.0,1.0,1.0,2.0,1.0,1.0,0.0,1.0,NaN,1.0,1.0,0.0,4.0,0.0,4.098210,NaN,0.0,0.0,0.0,0.0,NaN,1.0,1.0


##USING KNN FOR EFFICIENT DATA IMPUTATION

In [ ]:
!pip install impyute

In [ ]:
import tensorflow as tf
tf.test.gpu_device_name()

'/device:GPU:0'

In [ ]:
import sys
from impyute.imputation.cs import fast_knn
sys.setrecursionlimit(10000) #Increase the recursion limit of the OS

# start the KNN training
with tf.device('/GPU:0'):
  imputed_training=fast_knn(data[['Gender', 'Birth asphyxia']].values, k=30)

/usr/local/lib/python3.7/dist-packages/impyute/imputation/cs/fast_knn.py:113: RuntimeWarning: invalid value encountered in true_divide
  weights = distances/np.sum(distances)


In [ ]:
from sklearn.impute import KNNImputer

dv = data.drop("Patient Id", axis=1).values
imputer = KNNImputer()
imputer.fit(dv)
with tf.device('/GPU:0'):
  dvt = imputer.transform(dv)

In [ ]:
colNames = data.columns
patient = data["Patient Id"]
data = pd.DataFrame(dvt)
data.insert(0,"Patient Id", patient)
data.columns

Index(['Patient Id',            0,            1,            2,            3,
                  4,            5,            6,            7,            8,
                  9,           10,           11,           12,           13,
                 14,           15,           16,           17,           18,
                 19,           20,           21,           22,           23,
                 24,           25,           26,           27,           28,
                 29,           30,           31],
      dtype='object')

In [ ]:
data.columns = colNames
data.head()

,Patient Id,Patient Age,Genes in mother's side,Genes in father's side,Maternal gene,Paternal gene,Blood cell count (mcL),Mother's age,Father's age,Status,Respiratory Rate (breaths/min),Heart Rate (rates/min,Follow-up,Gender,Birth asphyxia,Place of birth,Folic acid details (peri-conceptional),H/O serious maternal illness,H/O radiation exposure (x-ray),H/O substance abuse,Assisted conception IVF/ART,History of anomalies in previous pregnancies,No. of previous abortion,Birth defects,White Blood cell count (thousand per microliter),Blood test result,Symptom 1,Symptom 2,Symptom 3,Symptom 4,Symptom 5,Genetic Disorder,Disorder Subclass
0,PID0x6418,2.0,1.0,0.0,1.0,0.0,4.760603,37.0,36.2,0.0,0.0,0.0,0.0,1.2,0.8,1.0,0.0,0.6,0.0,0.0,0.0,1.0,2.6,0.2,9.857562,1.2,1.0,1.0,1.0,1.0,1.0,0.0,5.0
1,PID0x25d5,4.0,1.0,1.0,0.0,0.0,4.910669,39.6,23.0,1.0,1.0,0.0,0.0,1.4,0.0,0.6,1.0,1.0,1.0,1.0,0.0,1.0,2.4,0.0,5.522560,1.0,1.0,0.8,1.0,1.0,0.0,2.0,2.0
2,PID0x4a82,6.0,1.0,0.0,0.0,0.0,4.893297,41.0,22.0,0.0,0.0,1.0,1.0,0.6,1.0,0.6,1.0,0.0,2.0,0.4,1.0,1.0,4.0,1.0,5.966703,1.0,0.0,1.0,1.0,1.0,1.0,1.0,3.0
3,PID0x4ac8,12.0,1.0,0.0,1.0,0.0,4.705280,21.0,35.4,1.0,1.0,0.0,0.0,2.0,1.0,1.0,0.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,7.919321,0.0,0.0,0.0,1.0,0.0,0.0,0.0,6.0
4,PID0x1bf7,11.0,1.0,0.0,0.4,1.0,4.720703,32.0,40.0,0.0,1.0,1.0,1.0,2.0,1.0,1.0,0.0,1.0,1.0,1.0,1.0,0.0,4.0,0.0,4.098210,1.6,0.0,0.0,0.0,0.0,0.4,1.0,1.0


In [ ]:
df_min_max_scaled.to_csv('/content/drive/MyDrive/GenDis/preproc3.csv')

In [ ]:
df = data.drop(["Patient Id", "Blood cell count (mcL)", "White Blood cell count (thousand per microliter)"], axis=1)
df = df.round()
df.head()

,Patient Age,Genes in mother's side,Genes in father's side,Maternal gene,Paternal gene,Mother's age,Father's age,Status,Respiratory Rate (breaths/min),Heart Rate (rates/min,Follow-up,Gender,Birth asphyxia,Place of birth,Folic acid details (peri-conceptional),H/O serious maternal illness,H/O radiation exposure (x-ray),H/O substance abuse,Assisted conception IVF/ART,History of anomalies in previous pregnancies,No. of previous abortion,Birth defects,Blood test result,Symptom 1,Symptom 2,Symptom 3,Symptom 4,Symptom 5,Genetic Disorder,Disorder Subclass
0,2.0,1.0,0.0,1.0,0.0,37.0,36.0,0.0,0.0,0.0,0.0,1.0,1.0,1.0,0.0,1.0,0.0,0.0,0.0,1.0,3.0,0.0,1.0,1.0,1.0,1.0,1.0,1.0,0.0,5.0
1,4.0,1.0,1.0,0.0,0.0,40.0,23.0,1.0,1.0,0.0,0.0,1.0,0.0,1.0,1.0,1.0,1.0,1.0,0.0,1.0,2.0,0.0,1.0,1.0,1.0,1.0,1.0,0.0,2.0,2.0
2,6.0,1.0,0.0,0.0,0.0,41.0,22.0,0.0,0.0,1.0,1.0,1.0,1.0,1.0,1.0,0.0,2.0,0.0,1.0,1.0,4.0,1.0,1.0,0.0,1.0,1.0,1.0,1.0,1.0,3.0
3,12.0,1.0,0.0,1.0,0.0,21.0,35.0,1.0,1.0,0.0,0.0,2.0,1.0,1.0,0.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,6.0
4,11.0,1.0,0.0,0.0,1.0,32.0,40.0,0.0,1.0,1.0,1.0,2.0,1.0,1.0,0.0,1.0,1.0,1.0,1.0,0.0,4.0,0.0,2.0,0.0,0.0,0.0,0.0,0.0,1.0,1.0


In [ ]:
df.insert(0,"White Blood cell count (thousand per microliter)", data["White Blood cell count (thousand per microliter)"])
df.insert(0,"Blood cell count (mcL)", data["Blood cell count (mcL)"])
df.insert(0,"Patient Id", data["Patient Id"])

In [ ]:
df.head()

,Patient Id,Blood cell count (mcL),White Blood cell count (thousand per microliter),Patient Age,Genes in mother's side,Genes in father's side,Maternal gene,Paternal gene,Mother's age,Father's age,Status,Respiratory Rate (breaths/min),Heart Rate (rates/min,Follow-up,Gender,Birth asphyxia,Place of birth,Folic acid details (peri-conceptional),H/O serious maternal illness,H/O radiation exposure (x-ray),H/O substance abuse,Assisted conception IVF/ART,History of anomalies in previous pregnancies,No. of previous abortion,Birth defects,Blood test result,Symptom 1,Symptom 2,Symptom 3,Symptom 4,Symptom 5,Genetic Disorder,Disorder Subclass
0,PID0x6418,4.760603,9.857562,2.0,1.0,0.0,1.0,0.0,37.0,36.0,0.0,0.0,0.0,0.0,1.0,1.0,1.0,0.0,1.0,0.0,0.0,0.0,1.0,3.0,0.0,1.0,1.0,1.0,1.0,1.0,1.0,0.0,5.0
1,PID0x25d5,4.910669,5.522560,4.0,1.0,1.0,0.0,0.0,40.0,23.0,1.0,1.0,0.0,0.0,1.0,0.0,1.0,1.0,1.0,1.0,1.0,0.0,1.0,2.0,0.0,1.0,1.0,1.0,1.0,1.0,0.0,2.0,2.0
2,PID0x4a82,4.893297,5.966703,6.0,1.0,0.0,0.0,0.0,41.0,22.0,0.0,0.0,1.0,1.0,1.0,1.0,1.0,1.0,0.0,2.0,0.0,1.0,1.0,4.0,1.0,1.0,0.0,1.0,1.0,1.0,1.0,1.0,3.0
3,PID0x4ac8,4.705280,7.919321,12.0,1.0,0.0,1.0,0.0,21.0,35.0,1.0,1.0,0.0,0.0,2.0,1.0,1.0,0.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,6.0
4,PID0x1bf7,4.720703,4.098210,11.0,1.0,0.0,0.0,1.0,32.0,40.0,0.0,1.0,1.0,1.0,2.0,1.0,1.0,0.0,1.0,1.0,1.0,1.0,0.0,4.0,0.0,2.0,0.0,0.0,0.0,0.0,0.0,1.0,1.0


In [ ]:
df.describe()

,Blood cell count (mcL),White Blood cell count (thousand per microliter),Patient Age,Genes in mother's side,Genes in father's side,Maternal gene,Paternal gene,Mother's age,Father's age,Status,Respiratory Rate (breaths/min),Heart Rate (rates/min,Follow-up,Gender,Birth asphyxia,Place of birth,Folic acid details (peri-conceptional),H/O serious maternal illness,H/O radiation exposure (x-ray),H/O substance abuse,Assisted conception IVF/ART,History of anomalies in previous pregnancies,No. of previous abortion,Birth defects,Blood test result,Symptom 1,Symptom 2,Symptom 3,Symptom 4,Symptom 5,Genetic Disorder,Disorder Subclass
count,22083.000000,22083.000000,22083.000000,22083.000000,22083.000000,22083.000000,22083.000000,22083.000000,22083.000000,22083.000000,22083.000000,22083.000000,22083.000000,22083.000000,22083.000000,22083.000000,22083.000000,22083.000000,22083.000000,22083.000000,22083.000000,22083.000000,22083.000000,22083.000000,22083.000000,22083.000000,22083.000000,22083.000000,22083.000000,22083.000000,22083.000000,22083.000000
mean,4.899004,7.485572,6.976543,0.614998,0.375628,0.575737,0.413169,34.546438,42.036952,0.472717,0.468233,0.462030,0.527465,0.995562,1.009872,0.530816,0.529412,0.474573,0.997238,1.002853,0.525200,0.532582,2.003532,0.528868,1.472264,0.618349,0.582031,0.561291,0.472218,0.434135,0.867998,5.286601
std,0.194918,2.486594,4.106806,0.486607,0.484296,0.494242,0.492414,8.497604,11.254762,0.499266,0.499001,0.498567,0.499256,0.769151,0.661676,0.499061,0.499146,0.499364,0.693783,0.699050,0.499376,0.498949,1.324556,0.499177,1.052936,0.485803,0.493236,0.496240,0.499239,0.495654,0.897677,2.026421
min,4.092727,3.000000,0.000000,0.000000,0.000000,0.000000,0.000000,18.000000,20.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000
25%,4.771418,5.705572,4.000000,0.000000,0.000000,0.000000,0.000000,29.000000,34.000000,0.000000,0.000000,0.000000,0.000000,0.000000,1.000000,0.000000,0.000000,0.000000,1.000000,1.000000,0.000000,0.000000,1.000000,0.000000,1.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,4.000000
50%,4.899004,7.484680,7.000000,1.000000,0.000000,1.000000,0.000000,35.000000,42.000000,0.000000,0.000000,0.000000,1.000000,1.000000,1.000000,1.000000,1.000000,0.000000,1.000000,1.000000,1.000000,1.000000,2.000000,1.000000,1.000000,1.000000,1.000000,1.000000,0.000000,0.000000,1.000000,6.000000
75%,5.025826,9.219289,10.000000,1.000000,1.000000,1.000000,1.000000,40.000000,50.000000,1.000000,1.000000,1.000000,1.000000,2.000000,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,3.000000,1.000000,2.000000,1.000000,1.000000,1.000000,1.000000,1.000000,2.000000,7.000000
max,5.609829,12.000000,14.000000,1.000000,1.000000,1.000000,1.000000,51.000000,64.000000,1.000000,1.000000,1.000000,1.000000,2.000000,2.000000,1.000000,1.000000,1.000000,2.000000,2.000000,1.000000,1.000000,4.000000,1.000000,3.000000,1.000000,1.000000,1.000000,1.000000,1.000000,2.000000,8.000000


##MIN MAX SCALING FOR NORMALIZATION OF CONTINUOUS DATA

In [ ]:
# from sklearn.preprocessing import MinMaxScaler
# scaler = MinMaxScaler()
# scaler.fit(df[['Blood cell count (mcL)']])
# df[['Blood cell count (mcL)']] = scaler.transform(df[['Blood cell count (mcL)']])
# scaled_features

# apply normalization techniques by Column 1
column='Father\'s age'
df_min_max_scaled[column]=(df_min_max_scaled[column]-df_min_max_scaled[column].min())/(df_min_max_scaled[column].max()-df_min_max_scaled[column].min())
# view normalized data
display(df_min_max_scaled)

,Patient Id,Blood cell count (mcL),White Blood cell count (thousand per microliter),Patient Age,Genes in mother's side,Genes in father's side,Maternal gene,Paternal gene,Mother's age,Father's age,Status,Respiratory Rate (breaths/min),Heart Rate (rates/min,Follow-up,Gender,Birth asphyxia,Place of birth,Folic acid details (peri-conceptional),H/O serious maternal illness,H/O radiation exposure (x-ray),H/O substance abuse,Assisted conception IVF/ART,History of anomalies in previous pregnancies,No. of previous abortion,Birth defects,Blood test result,Symptom 1,Symptom 2,Symptom 3,Symptom 4,Symptom 5,Genetic Disorder,Disorder Subclass
0,PID0x6418,0.440231,0.761951,0.142857,1.0,0.0,1.0,0.0,0.575758,0.363636,0.0,0.0,0.0,0.0,1.0,1.0,1.0,0.0,1.0,0.0,0.0,0.0,1.0,3.0,0.0,1.0,1.0,1.0,1.0,1.0,1.0,0.0,5.0
1,PID0x25d5,0.539148,0.280284,0.285714,1.0,1.0,0.0,0.0,0.666667,0.068182,1.0,1.0,0.0,0.0,1.0,0.0,1.0,1.0,1.0,1.0,1.0,0.0,1.0,2.0,0.0,1.0,1.0,1.0,1.0,1.0,0.0,2.0,2.0
2,PID0x4a82,0.527697,0.329634,0.428571,1.0,0.0,0.0,0.0,0.696970,0.045455,0.0,0.0,1.0,1.0,1.0,1.0,1.0,1.0,0.0,2.0,0.0,1.0,1.0,4.0,1.0,1.0,0.0,1.0,1.0,1.0,1.0,1.0,3.0
3,PID0x4ac8,0.403765,0.546591,0.857143,1.0,0.0,1.0,0.0,0.090909,0.340909,1.0,1.0,0.0,0.0,2.0,1.0,1.0,0.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,6.0
4,PID0x1bf7,0.413931,0.122023,0.785714,1.0,0.0,0.0,1.0,0.424242,0.454545,0.0,1.0,1.0,1.0,2.0,1.0,1.0,0.0,1.0,1.0,1.0,1.0,0.0,4.0,0.0,2.0,0.0,0.0,0.0,0.0,0.0,1.0,1.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
22078,NaN,0.531459,0.498298,0.500000,1.0,0.0,1.0,0.0,0.515152,0.500000,0.0,0.0,0.0,1.0,1.0,1.0,1.0,1.0,0.0,1.0,1.0,1.0,1.0,2.0,1.0,1.0,1.0,1.0,1.0,0.0,0.0,1.0,5.0
22079,NaN,0.531459,0.498298,0.500000,1.0,0.0,1.0,0.0,0.515152,0.500000,0.0,0.0,0.0,1.0,1.0,1.0,1.0,1.0,0.0,1.0,1.0,1.0,1.0,2.0,1.0,1.0,1.0,1.0,1.0,0.0,0.0,1.0,5.0
22080,NaN,0.531459,0.498298,0.500000,1.0,0.0,1.0,0.0,0.515152,0.500000,0.0,0.0,0.0,1.0,1.0,1.0,1.0,1.0,0.0,1.0,1.0,1.0,1.0,2.0,1.0,1.0,1.0,1.0,1.0,0.0,0.0,1.0,5.0
22081,NaN,0.531459,0.498298,0.500000,1.0,0.0,1.0,0.0,0.515152,0.500000,0.0,0.0,0.0,1.0,1.0,1.0,1.0,1.0,0.0,1.0,1.0,1.0,1.0,2.0,1.0,1.0,1.0,1.0,1.0,0.0,0.0,1.0,5.0


In [ ]:
df_min_max_scaled.describe()

,Blood cell count (mcL),White Blood cell count (thousand per microliter),Patient Age,Genes in mother's side,Genes in father's side,Maternal gene,Paternal gene,Mother's age,Father's age,Status,Respiratory Rate (breaths/min),Heart Rate (rates/min,Follow-up,Gender,Birth asphyxia,Place of birth,Folic acid details (peri-conceptional),H/O serious maternal illness,H/O radiation exposure (x-ray),H/O substance abuse,Assisted conception IVF/ART,History of anomalies in previous pregnancies,No. of previous abortion,Birth defects,Blood test result,Symptom 1,Symptom 2,Symptom 3,Symptom 4,Symptom 5,Genetic Disorder,Disorder Subclass
count,22083.000000,22083.000000,22083.000000,22083.000000,22083.000000,22083.000000,22083.000000,22083.000000,22083.000000,22083.000000,22083.000000,22083.000000,22083.000000,22083.000000,22083.000000,22083.000000,22083.000000,22083.000000,22083.000000,22083.000000,22083.000000,22083.000000,22083.000000,22083.000000,22083.000000,22083.000000,22083.000000,22083.000000,22083.000000,22083.000000,22083.000000,22083.000000
mean,0.531459,0.498397,0.498325,0.614998,0.375628,0.575737,0.413169,0.501407,0.500840,0.472717,0.468233,0.462030,0.527465,0.995562,1.009872,0.530816,0.529412,0.474573,0.997238,1.002853,0.525200,0.532582,2.003532,0.528868,1.472264,0.618349,0.582031,0.561291,0.472218,0.434135,0.867998,5.286601
std,0.128480,0.276288,0.293343,0.486607,0.484296,0.494242,0.492414,0.257503,0.255790,0.499266,0.499001,0.498567,0.499256,0.769151,0.661676,0.499061,0.499146,0.499364,0.693783,0.699050,0.499376,0.498949,1.324556,0.499177,1.052936,0.485803,0.493236,0.496240,0.499239,0.495654,0.897677,2.026421
min,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000
25%,0.447360,0.300619,0.285714,0.000000,0.000000,0.000000,0.000000,0.333333,0.318182,0.000000,0.000000,0.000000,0.000000,0.000000,1.000000,0.000000,0.000000,0.000000,1.000000,1.000000,0.000000,0.000000,1.000000,0.000000,1.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,4.000000
50%,0.531459,0.498298,0.500000,1.000000,0.000000,1.000000,0.000000,0.515152,0.500000,0.000000,0.000000,0.000000,1.000000,1.000000,1.000000,1.000000,1.000000,0.000000,1.000000,1.000000,1.000000,1.000000,2.000000,1.000000,1.000000,1.000000,1.000000,1.000000,0.000000,0.000000,1.000000,6.000000
75%,0.615054,0.691032,0.714286,1.000000,1.000000,1.000000,1.000000,0.666667,0.681818,1.000000,1.000000,1.000000,1.000000,2.000000,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,3.000000,1.000000,2.000000,1.000000,1.000000,1.000000,1.000000,1.000000,2.000000,7.000000
max,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,2.000000,2.000000,1.000000,1.000000,1.000000,2.000000,2.000000,1.000000,1.000000,4.000000,1.000000,3.000000,1.000000,1.000000,1.000000,1.000000,1.000000,2.000000,8.000000


In [ ]:
  print("UNIQUE VALUES IN EACH COLUMN")
for i in data.columns:
    print(i)
    print(data[i].unique())
    print(" ")

UNIQUE VALUES IN EACH COLUMN
Patient Id
['PID0x6418' 'PID0x25d5' 'PID0x4a82' ... 'PID0x8d32' 'PID0x883d' nan]
 
Patient Age
[ 2.  4.  6. 12. 11. 14.  3.  7.  1.  0. nan 10.  5. 13.  8.  9.]
 
Genes in mother's side
['Yes' 'No' nan]
 
Inherited from father
['No' 'Yes' nan]
 
Maternal gene
['Yes' 'No' nan]
 
Paternal gene
['No' 'Yes' nan]
 
Blood cell count (mcL)
[4.76060309 4.91066906 4.89329743 ... 5.22482777 5.13794212        nan]
 
Mother's age
[nan 41. 21. 32. 40. 45. 44. 50. 28. 30. 24. 36. 51. 23. 49. 46. 18. 38.
 37. 42. 48. 25. 19. 47. 34. 35. 22. 33. 20. 26. 31. 29. 27. 43. 39.]
 
Father's age
[nan 23. 22. 63. 44. 42. 56. 20. 24. 57. 48. 30. 55. 62. 43. 32. 41. 52.
 28. 31. 61. 35. 49. 50. 29. 64. 39. 34. 51. 25. 60. 53. 58. 26. 27. 59.
 47. 38. 54. 21. 37. 36. 33. 46. 40. 45.]
 
Status
['Alive' 'Deceased' nan]
 
Respiratory Rate (breaths/min)
['Normal (30-60)' 'Tachypnea' nan]
 
Heart Rate (rates/min
['Normal' 'Tachycardia' nan]
 
Follow-up
['High' 'Low' nan]
 
Gender
[nan 'Ma

In [ ]:
data.isnull().sum()

In [ ]:
data.dropna(inplace=True)

In [ ]:
data.info()

In [ ]:
data.shape

In [ ]:
from sklearn.model_selection import train_test_split
dfTrain, dfTest = train_test_split(df_min_max_scaled, test_size=0.3)

In [ ]:
df_min_max_scaled['Genetic Disorder'].value_counts() / df_min_max_scaled.shape[0]

0.0    0.477607
2.0    0.345605
1.0    0.176788
Name: Genetic Disorder, dtype: float64

In [ ]:
dfTest['Genetic Disorder'].value_counts() / dfTest.shape[0]

0.0    0.473811
2.0    0.344302
1.0    0.181887
Name: Genetic Disorder, dtype: float64

##SAVING THE FILE

In [ ]:
dfTrain.to_csv('/content/drive/MyDrive/GenDis/preproctrain.csv')
dfTest.to_csv('/content/drive/MyDrive/GenDis/preproctest.csv')